# KNN

### Importação das bibliotecas necessárias

In [ ]:
import pandas as pd
from random import randint
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import itertools
from joblib import dump, load
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier  


### Carregamento da base

In [ ]:
#NomeBase = 'LBP10r_1_5_p_12'
#NomeBase = 'LBP20r_1_5_p_12'
#NomeBase = 'LBP30r_1_5_p_12'
#NomeBase = 'LBP40r_1_5_p_12'
#NomeBase = 'LBP50r_1_5_p_12'
#NomeBase = 'LBP75r_1_5_p_12'
#NomeBase = 'LBP100r_1_5_p_12'

#NomeBase = 'LBP10r_3_p_24'
#NomeBase = 'LBP20r_3_p_24'
#NomeBase = 'LBP30r_3_p_24'
#NomeBase = 'LBP40r_3_p_24'
#NomeBase = 'LBP50r_3_p_24'
NomeBase = 'GLCMMaior'





valor = 1
base = pd.read_csv('../Datasets/' + str(NomeBase)+'.txt', header=None)
entradas = base.iloc[:, 0:len(base.columns) -valor].values 

classes = base.iloc[:,len(base.columns)-valor].values

NumColunas = len(base.columns) - valor
caminho = '../resultados/KNN/PesosKNN_Base'+str(NomeBase)
print(classes.shape)

### Padronização dos dados

In [ ]:
for i in range (len(base.columns)-valor):
    entradas[:,i] = (entradas[:,i] - np.mean(entradas[:,i]))/entradas[:,i].std(axis = 0)

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(entradas)
entradas = scaler.transform(entradas) 

entradas_min = entradas.min()
entradas_max = entradas.max()
entradas_range = (entradas_max - entradas_min)
entradas = (entradas - entradas_min)/(entradas_range)

In [ ]:
labelEncoder = LabelEncoder()
classes = labelEncoder.fit_transform(classes)

### Divisão da base em treinamento (80%) e teste (20%)

In [ ]:
def dividir_trei_test(entradas, classes, percentual):
    result = []
    while (len(result)) != len(entradas):
        r = randint(0, len(entradas)-1)
        if r not in result:
            result.append(r)    

    repeticoes = np.zeros(np.max(classes)+1)
    
    for i in range(len(entradas)):
        repeticoes[classes[i]] = repeticoes[classes[i]] + 1
    repeticoes = np.array(repeticoes)
    minimo = int(np.min(repeticoes) * (1 - percentual))
    
    treino = int((minimo) * (np.max(classes)+1) )
    teste = int(len(entradas) - treino)

    entradas_trei = np.zeros((treino, NumColunas))
    entradas_test = np.zeros((teste, NumColunas))
    classes_trei = np.zeros((treino)) 
    classes_test = np.zeros((teste))
    classeErro_trei = np.zeros((treino))
    classeErro_test = np.zeros((teste))
 
    contTreino = 0
    contTest = 0
    vetorMinimo = np.zeros(np.max(classes)+1)

    for i in range(len(classes)):    
        if (vetorMinimo[classes[result[i]]] < minimo):            
            entradas_trei[contTreino] = entradas[result[i]]
            classes_trei[contTreino] = classes[result[i]]
          #  classeErro_trei[contTreino] = classesErro[result[i]]
            contTreino = contTreino + 1
            vetorMinimo[classes[result[i]]] = vetorMinimo[classes[result[i]]] + 1
            
        else:   
            entradas_test[contTest] = entradas[result[i]]            
            classes_test[contTest] = classes[result[i]]
          #  classeErro_test[contTest] = classesErro[result[i]]
            contTest = contTest + 1        


    return entradas_trei, entradas_test, classes_trei, classes_test

In [ ]:
labelEncoder = LabelEncoder()
classes = labelEncoder.fit_transform(classes)
NumClasses = np.max(classes) + 2

entradas_trei, entradas_test, classes_t, classes_te = dividir_trei_test(entradas, classes, 0.2)

## KNN


In [ ]:
from sklearn.model_selection import cross_val_score
#carregarPesos = int(input("Deseja carregar um treinamento? (1 - Sim, 2 - Não) "))
carregarPesos = 2
if (carregarPesos != 1):        
    kf = KFold(n_splits=10)
    vizinhos = int(input("Digite a quantidade de vizinhos: "))
    classificador = KNeighborsClassifier(n_neighbors=vizinhos, metric='chebyshev')  
      
 
    cross_val_score(classificador, entradas_trei, classes_t, scoring='accuracy', cv=10).mean()

if (carregarPesos == 1):
    numClass = int(input("Digite o número do diretório que deseja carregar"))
    classificador = load(str(caminho)+'/'+str(numClass)+'/classificadorKNN.joblib') 
   

In [ ]:
print(classificador)

In [ ]:
classificador.fit(entradas_trei, classes_t)
ACtreino = (classificador.score(entradas_trei, classes_t)) * 100
ACTreiE = 100 - ACtreino 

previsoesTrei = classificador.predict(entradas_trei)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de Confusão',
                          cmap=plt.cm.Blues):   

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Saída correta')
    plt.xlabel('Saída encontrada')    

In [ ]:
cm = confusion_matrix(classes_t, previsoesTrei)
plot_confusion_matrix(cm, 
                      classes=['0', '1', 'FR'],
                      title='Treinamento - KNN')
plt.show()

In [ ]:
contT = 0
errosTrei = []
errosTreiC = []
for i in range (len(classes_t)):
    if (classes_t[i] != previsoesTrei[i] and previsoesTrei[i] != 2):
        contT += 1
        errosTrei.append(classes_t[i])
        #errosTreiC.append(classeErro_trei[i])
        
erro = (contT * 100)/len(classes_t)

In [ ]:
Ac0 = precision_recall_fscore_support(classes_t,previsoesTrei, average=None)[0][0]
Ac1 = precision_recall_fscore_support(classes_t,previsoesTrei, average=None)[0][1]

print("Acurácia nas peles boas: " + str(Ac0))
print("Acurácia nas peles ruins: " + str(Ac1))


### Testes

In [ ]:
previsoes = classificador.predict(entradas_test)

### Salvar o modelo

In [ ]:
if (carregarPesos != 1):
    import os
    if not os.path.exists(str(caminho)+"/"):
        os.mkdir(str(caminho))      
    
    valor  = 0
 

    if not os.path.exists(str(caminho)+"/"+str(int(valor))):
        os.mkdir(str(caminho)+ "/" + str(int(valor)))    

    else:       
        b = os.listdir(caminho)
        #print(b)
        a = np.zeros(len(b))

        for i in range(len(b)):
            a[i] = int(b[i])

        valor = np.max(a)
        valor = valor + 1

        os.mkdir(str(caminho) + "/"+ str(int(valor)))
  
    dump(classificador, str(caminho) + '/'+str(int(valor))+'/classificadorKNN.joblib') 

### Matriz de confusão

In [ ]:
cm = confusion_matrix(classes_te, previsoes)
plot_confusion_matrix(cm, 
                      classes=['0', '1', 'FR'],
                      title='Teste - KNN')
plt.show()

In [ ]:
cont = 0
errosTest = []
errosTestC = []
for i in range (len(classes_te)):
    if (classes_te[i] != previsoes[i] and previsoes[i] != 2):
        cont += 1
        errosTest.append(classes_te[i])
       # errosTestC.append(classeErro_test[i])
        
erro = (cont * 100)/len(classes_te)

In [ ]:
print(classification_report(classes_te,previsoes))  



Ac0T = precision_recall_fscore_support(classes_te,previsoes, average=None)[0][0]
Ac1T = precision_recall_fscore_support(classes_te,previsoes, average=None)[0][1]

print("Acurácia nas peles boas: " + str(Ac0T))
print("Acurácia nas peles ruins: " + str(Ac1T))


In [ ]:
if (carregarPesos != 1):  
    caminhoTexto = '../resultados/KNN/'+str(NomeBase)+'Dados.txt'
    
    if not os.path.exists(caminhoTexto):
        arquivo = open(caminhoTexto, 'w')
        arquivo.writelines("vizinhos, Trei0, Trei1, Test0, Test1, erroTrei, erroTest, acertoTrei, acertoTest")
        arquivo.close()
        
    
    
    
    
    arquivo = open(caminhoTexto, 'r')
    conteudo = arquivo.readlines()
        
    conteudo.append("\n"+ str(vizinhos)+", "+ str(Ac0)+", "+ str(Ac1)+", "+ str(Ac0T)+", "+ str(Ac1T)+", " + str(ACTreiE)+", "+ str(erro)+", "+ str(ACtreino)+ ", " + str(100 - erro))

    #### Abre o arquivo como escrita, escreve o conteúdo e fecha o mesmo
        
    arquivo = open('../resultados/KNN/'+str(NomeBase)+'Dados.txt', 'w')
    arquivo.writelines(conteudo)
    arquivo.close()

In [ ]:

print('../resultados/KNN/'+str(NomeBase)+'Dados.txt')

arquivo = pd.read_csv('../resultados/KNN/'+str(NomeBase)+'Dados.txt')
arquivo = arquivo.sort_values(by=[' acertoTest', ' acertoTrei'], ascending=False)
print("10 Melhores Resultados - Base "+ str(NomeBase))

arquivo.head(15)

In [ ]:
print("Treino")
print("Quantidade de erros "+ str(contT))
print("Total " + str(len(classes_t)))
print("Erros Normal e Falha" )
print({x:errosTrei.count(x) for x in set(errosTrei)})


print("")
print("Teste")
print("Quantidade de erros "+ str(cont))
print("Total " + str(len(classes_te)))
print("Erros Normal e Falha" )
print({x:errosTest.count(x) for x in set(errosTest)})


print("Treinamento:")
print("Percentual de acerto  "+ str(ACtreino))
print("Percentual de erro "+ str(ACTreiE))

print("Teste:")
print("Percentual de acerto  "+ str(100 - erro))
print("Percentual de erro "+ str(erro))